In [ ]:
!========================================================================Training Model=====================================================================

Import library

In [ ]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

Connect to GG drive

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Training keras model

In [ ]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('/content/drive/MyDrive/Colab/intents_eng.json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.summary()

model.fit(trainX, trainY, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model_eng.h5')
print('Done')

Convert h5 model to tflite model

In [ ]:
# Convert h5 to tflite model
model = tf.keras.models.load_model('/content/chatbot_model_eng.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("chatbot_model_eng.tflite", "wb").write(tflite_model)

Using tflite model

In [18]:
interpreter = tf.lite.Interpreter(model_path="/content/chatbot_model_eng.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('/content/drive/MyDrive/Colab/intents_eng.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

def clean_up_sentence(sentence):
        sentence_words = nltk.word_tokenize(sentence)
        sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words] #jjj
        return sentence_words

def bag_of_words(sentence):
        sentence_words = clean_up_sentence(sentence)
        bag = [0] * len(words)
        for w in sentence_words:
            for i, word in enumerate(words):
                if word == w:
                    bag[i] = 1
        return np.array(bag)

def predict_class(sentence):
        bow = bag_of_words(sentence)

        # using tflite model
        input_shape = input_details[0]['shape']
        input_data = np.array(np.array([bow]), dtype=np.float32)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        res = interpreter.get_tensor(output_details[0]['index'])[0]

        ERROR_THRESHOLD = 0.25
        results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
        return return_list

def get_response(intents_list, intents_json):
        try:
            tag = intents_list[0]['intent']
            list_of_intents = intents_json['intents']
            for i in list_of_intents:
                if i['tag']  == tag:
                    result = random.choice(i['responses'])
                    break
        except IndexError:
            result = "I don't understand!"
        return result

In [20]:
print("Chatbot is running!")

# happiness - surprise - sadness - anger - disgust - fear - contempt

while True:
  message = input("")
  intents_list = predict_class(message)
  res = get_response(intents_list, intents)
  print(res)

Chatbot is running!
happiness
May your happiness continue to radiate and inspire. Let's enjoy some soulful music together!
surprise
Surprises make life interesting and keep us on our toes. Let's celebrate with some lively music!
sadness
In times of sadness, music becomes a source of solace and strength. Let it embrace and uplift your spirit.
anger
Take a deep breath and let the music guide you towards a more peaceful state of mind. We'll work through this anger together.
disgust
I'm sorry if something has disgusted you. Is there anything specific you'd like to discuss or address? Let's listen to some music to shift the focus.
fear
It's normal to experience fear, but it shouldn't hold you back. Let's face your fears head-on while music empowers and encourages you.
contempt
Contempt can be a strong emotion. Let's try to understand the root cause and find a resolution. Music can create a harmonious environment for discussion.


KeyboardInterrupt: ignored

Using h5 model

In [5]:
from tensorflow.keras.models import load_model

In [6]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('/content/drive/MyDrive/Colab/intents_eng.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

model = load_model('/content/chatbot_model_eng.h5')

def clean_up_sentence(sentence):
        sentence_words = nltk.word_tokenize(sentence)
        sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words] #jjj
        return sentence_words

def bag_of_words(sentence):
        sentence_words = clean_up_sentence(sentence)
        bag = [0] * len(words)
        for w in sentence_words:
            for i, word in enumerate(words):
                if word == w:
                    bag[i] = 1
        return np.array(bag)

def predict_class(sentence):
        bow = bag_of_words(sentence)
        res = model.predict(np.array([bow]))[0]
        ERROR_THRESHOLD = 0.25
        results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
        return return_list

def get_response(intents_list, intents_json):
        try:
            tag = intents_list[0]['intent']
            list_of_intents = intents_json['intents']
            for i in list_of_intents:
                if i['tag']  == tag:
                    result = random.choice(i['responses'])
                    break
        except IndexError:
            result = "I don't understand!"
        return result

In [7]:
print("Chatbot is running!")

while True:
  message = input("")
  intents_list = predict_class(message)
  res = get_response(intents_list, intents)
  print(res)

Chatbot is running!
happinness
1/1 [==============================] - 0s 149ms/step
I don't understand!
sadness
1/1 [==============================] - 0s 21ms/step
In times of sadness, music becomes a source of solace and strength. Let it embrace and uplift your spirit.


KeyboardInterrupt: ignored